In [2]:
### loading libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [29]:
# present working directory
%pwd
# changing directory to the data folder
%cd /uufs/chpc.utah.edu/common/home/gompert-group4/projects/fluctCNN/CNN_data2

In [3]:
import os
directory_path ='/uufs/chpc.utah.edu/common/home/gompert-group4/projects/fluctCNN/CNN_data2'

In [21]:
### making a list of all csv files
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]
#print(len(csv_files))

In [20]:
# read the list of csv files into pandas dataframe
df=pd.read_csv(csv_files[1], header=None, sep=' ', dtype='float32')
#df

In [11]:
### processing the list of csv files to make arrays of input matrices and extracting output labels 
# Load the CSV matrices from dataframe to numpy arrays
matrices = []
out_vec = []
for file in csv_files:
    df = pd.read_csv(file, header=None, sep=' ', dtype='float32')
    #df = df.iloc[1:, 1:].values
    #df = df.astype('float32')
    matrix = df
    matrices.append(matrix)
    # appending first five letters to out_vec as labels
    label = file[:5]
    out_vec.append(label)
input_arrays = np.stack(matrices)
# reshaping input data arrays
input_arr = np.reshape(input_arrays, (60000, 10, 20, 1))
input_arr.shape

In [22]:
### Processing Labels
labels =out_vec
label_mapping = {label: index for index, label in enumerate(set(labels))}
#print(label_mapping)
integer_labels = np.array([label_mapping[label] for label in labels])
categorical_labels = tf.one_hot(integer_labels, depth=len(label_mapping))
out_labels = np.stack(categorical_labels)
#out_labels.shape

In [19]:
### dividing input and label data into training, testingsets
input_train, input_test, output_train, output_test = train_test_split(input_arr, out_labels, test_size=0.2, random_state=100)

In [ ]:
### CNN
model = keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(4, 1), strides=(2, 2), padding ="same", activation="relu", input_shape=(10, 20, 1)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(128, kernel_size=(2, 2), activation="relu"))
#model.add(layers.Dropout(0.25)) 
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(2, activation="softmax"))

optimizer = keras.optimizers.Adam(learning_rate=0.0001)
# Compile the model
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(input_train, output_train, batch_size=32, epochs=10)#, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(input_test, output_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)